In [1]:
!pip3 install minio boto3==1.9.240

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import boto3
import json
from minio import Minio
from minio.error import ResponseError

In [3]:
S3_HOST=os.getenv('S3_HOST')
S3_ACCESS_KEY=os.getenv('S3_ACCESS_KEY')
S3_SECRET_KEY=os.getenv('S3_SECRET_KEY')
EXPERIMENT=os.getenv('EXPERIMENT')

In [4]:
policy = {"Version":"2012-10-17",
        "Statement":[
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetBucketLocation",
            "Resource":"arn:aws:s3:::mlflowruns"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:ListBucket",
            "Resource":"arn:aws:s3:::{}".format(EXPERIMENT)
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetObject",
            "Resource":"arn:aws:s3:::mlflowruns/*"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:PutObject",
            "Resource":"arn:aws:s3:::mlflowruns/*"
            }

        ]}


In [5]:
def create_s3_bucket():
    minioClient = Minio(S3_HOST,
                  access_key=S3_ACCESS_KEY,
                  secret_key=S3_SECRET_KEY,
                  secure=False)

    print(minioClient.list_buckets())

    try:
        if not minioClient.bucket_exists("mlflowruns"):
            minioClient.make_bucket('mlflowruns')
    except ResponseError as err:
        print(err)

    buckets = minioClient.list_buckets()
    for bucket in buckets:
        print(bucket.name, bucket.creation_date)

    policy = {"Version":"2012-10-17",
        "Statement":[
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetBucketLocation",
            "Resource":"arn:aws:s3:::mlflowruns"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:ListBucket",
            "Resource":"arn:aws:s3:::mlflowruns"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:GetObject",
            "Resource":"arn:aws:s3:::mlflowruns/*"
            },
            {
            "Sid":"",
            "Effect":"Allow",
            "Principal":{"AWS":"*"},
            "Action":"s3:PutObject",
            "Resource":"arn:aws:s3:::mlflowruns/*"
            }

        ]}

    minioClient.set_bucket_policy('mlflowruns', json.dumps(policy))

    # List all object paths in bucket that begin with my-prefixname.
    objects = minioClient.list_objects('mlflowruns', prefix='',
                              recursive=True)
    for obj in objects:
        print(obj.bucket_name, obj.object_name.encode('utf-8'), obj.last_modified,
            obj.etag, obj.size, obj.content_type)

In [6]:
create_s3_bucket()

[<minio.definitions.Bucket object at 0x7f78b0761790>, <minio.definitions.Bucket object at 0x7f78b0761750>]
mlflowruns 2021-02-04 19:05:53.508000+00:00
test 2021-02-02 14:13:35.144000+00:00
mlflowruns b'28ee189c162949abbaa64eec55bd905b/artifacts/model/MLmodel' 2021-02-04 19:05:59.748000+00:00 e0a7ff525cdc70acd650ea91da054877 334 None
mlflowruns b'28ee189c162949abbaa64eec55bd905b/artifacts/model/conda.yaml' 2021-02-04 19:05:59.768000+00:00 5598bd313c9c3bce2fc3b45242a6c3a9 142 None
mlflowruns b'28ee189c162949abbaa64eec55bd905b/artifacts/model/data/keras_module.txt' 2021-02-04 19:05:59.908000+00:00 65daa80bd048be9a6b0b2f1ba806122a 16 None
mlflowruns b'28ee189c162949abbaa64eec55bd905b/artifacts/model/data/model.h5' 2021-02-04 19:05:59.888000+00:00 5137d36ea0c0909f4bf2577091ec5256 237336 None
mlflowruns b'28ee189c162949abbaa64eec55bd905b/artifacts/model/data/save_format.txt' 2021-02-04 19:05:59.928000+00:00 77230e94b5cd88d51a872031531bfecd 2 None
mlflowruns b'28ee189c162949abbaa64eec55bd905b